In [47]:
from pyspark.sql import functions as F
from pyspark.sql.functions import concat, col, lit, split

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName('Construct Spark Graph')\
  .getOrCreate()

In [24]:
import os
import glob
import pandas as pd
os.chdir("data/doras/")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [26]:
df_dcu = spark.read.csv("data/doras/combined_csv.csv", header=True)

In [29]:
df_sub = df_dcu['Publication Title', 'Uncontrolled keywords']

In [30]:
df_sub.printSchema()

root
 |-- Publication Title: string (nullable = true)
 |-- Uncontrolled keywords: string (nullable = true)



In [37]:
df_dup_drop = df_sub.dropDuplicates()

In [38]:
df_dup_drop.toPandas()

,Publication Title,Uncontrolled keywords
0,Dublin City University participation in the VT...,Video captions
1,Automated murmurs: the social mobile tourist a...,social networks;
2,The ADAPT bilingual document alignment system ...,None
3,DCU at MMM 2013 video browser showdown.,video browser showdown
4,TennisSense: a multi-sensory approach to perfo...,sensors; tennis;
...,...,...
1469,Mulsemedia in telecommunication and networking...,Mulsemedia; Technology Enhanced Learning;
1470,Part-of-speech tagging of code-mixed social me...,None
1471,Urban enterprise: a review of smart city frame...,Smart City; Enterprise Architecture; Enterpris...
1472,An industrial case study to improve the onboar...,None


In [39]:
df_dup_drop = df_dup_drop.filter(df_dup_drop['Uncontrolled keywords'] != 'None')

In [40]:
#df_dup_drop.toPandas()
df_dup_drop = df_dup_drop.withColumn("Uncontrolled keywords", trim(df['Uncontrolled keywords']))

,Publication Title,Uncontrolled keywords
0,Dublin City University participation in the VT...,Video captions
1,Automated murmurs: the social mobile tourist a...,social networks;
2,DCU at MMM 2013 video browser showdown.,video browser showdown
3,TennisSense: a multi-sensory approach to perfo...,sensors; tennis;
4,A methodology for validating diversity in synt...,Time series; Forecasting; Time Series Predicti...
...,...,...
926,Nonlinearity reduction of manifolds using Gaus...,Computer vision; Principal Component Analysis;...
927,Overview of NTCIR-13 Lifelog-2 Task.,lifelog; test collection; information retrieva...
928,Mulsemedia in telecommunication and networking...,Mulsemedia; Technology Enhanced Learning;
929,Urban enterprise: a review of smart city frame...,Smart City; Enterprise Architecture; Enterpris...


In [78]:
keywords = []
#split_col = pyspark.sql.functions.split(df_dup_drop['Uncontrolled keywords'], ';')
dataCollect = df_dup_drop.select(split(df_dup_drop['Uncontrolled keywords'], ';')).collect()
'''
for keyword in dataCollect:
    if str(keyword) != 'None':
        for category in df_dup_drop.select(split(df_dup_drop['Uncontrolled keywords'], ';')).collect():
            #category = category.strip()
            if category not in keywords:
                keywords.append(category)
'''


"\nfor keyword in dataCollect:\n    if str(keyword) != 'None':\n        for category in df_dup_drop.select(split(df_dup_drop['Uncontrolled keywords'], ';')).collect():\n            #category = category.strip()\n            if category not in keywords:\n                keywords.append(category)\n"

In [65]:
#df_dup_drop.select(split(df_dup_drop['Uncontrolled keywords'], ';')).collect()

In [84]:
dataCollect[3]

Row(split(Uncontrolled keywords, ;, -1)=['sensors', ' tennis', ''])

In [85]:
len(dataCollect[3])

1